In [ ]:
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.pipeline import make_pipeline
from sklearn.experimental import enable_halving_search_cv

from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
X_test = pd.read_csv("/content/data_inlf_test.csv")
train = pd.read_csv("/content/data_inlf_train.csv")

In [ ]:
y_train=train["inlf"]
X_train=train.drop(columns="inlf")

In [ ]:
pipe_svc = make_pipeline(StandardScaler(),
      SVC(random_state=1))
param_range = [0.0001, 0.001, 0.01, 0.1,
      1.0, 10.0, 100.0, 1000.0]
param_grid = [{'svc__C': param_range,
      'svc__kernel': ['linear']},
      {'svc__C': param_range,
      'svc__gamma': param_range,
      'svc__kernel': ['rbf']}]
gs = GridSearchCV(estimator=pipe_svc,
      param_grid=param_grid,
      scoring='accuracy',
      cv=10,
      refit=True,
      n_jobs=-1)
gs = gs.fit(X_train, y_train)

In [ ]:
y_pred = gs.predict(X_test)

In [ ]:
first = pd.DataFrame(data = y_pred, index = range(189), columns = ["Expected"])
first.reset_index().rename (columns = {"index":"id"}).to_csv("first.csv", index = False)

In [ ]:
# Ampliar el rango de valores para los parámetros C y gamma
param_range = [0.0001, 0.001, 0.01, 0.1,
      1.0, 10.0, 100.0, 1000.0]

# Utilizar una estrategia de búsqueda más exhaustiva
param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']},
      {'svc__C': param_range,'svc__gamma': param_range,'svc__kernel': ['rbf']}]


# Experimentar con diferentes métricas de puntuación (puedes cambiar 'accuracy' por otras métricas)
scoring = ['accuracy', 'precision', 'recall', 'f1']

pipe_svc = make_pipeline(StandardScaler(), SVC(random_state=5))

# Realizar la búsqueda en cuadrícula con todas las combinaciones posibles
gs = GridSearchCV(
    estimator=pipe_svc,
    param_grid=param_grid,
    scoring=scoring,
    cv=7,
    refit="p",  # Refit con respecto a la precisión para obtener el mejor modelo
    n_jobs=-1
)

gs.fit(X_train, y_train)

# Obtener los mejores parámetros
best_params = gs.best_params_
print("Mejores parámetros:", best_params)

# Obtener el mejor modelo
best_model = gs.best_estimator_

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Crear un DataFrame con las predicciones
result_df = pd.DataFrame(data=y_pred, columns=["Expected"])
result_df.index.name = "id"

# Guardar las predicciones en un archivo CSV
result_df.to_csv("best_svm_predictions.csv")

# También puedes acceder a las métricas de rendimiento, como las puntuaciones de precisión, recall, f1, etc., si es necesario.

Mejores parámetros: {'svc__C': 1.0, 'svc__kernel': 'linear'}
